# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [96]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
from pathlib import Path

# Import API key
from api_keys import geoapify_key

import warnings
warnings.filterwarnings("ignore")

In [97]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
#city_data_df.dropna()
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kununurra,-15.7667,128.7333,28.99,26,0,7.20,AU,22-07-2023
1,1,port-aux-francais,-49.3500,70.2167,1.21,88,100,5.80,TF,22-07-2023
2,2,albany,42.6001,-73.9662,19.24,92,0,2.89,US,22-07-2023
3,3,adamstown,-25.0660,-130.1015,21.00,71,14,13.44,PN,22-07-2023
4,4,petropavlovsk-kamchatsky,53.0452,158.6483,22.33,60,75,4.00,RU,22-07-2023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [98]:
%%capture --no-display

# Configure the map plot
first_map = city_data_df.hvplot.points(
   "Lng",
   "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    color="City"
)

# Display the map
first_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [99]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df=city_data_df.loc[(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]<27) & (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
#city_data_df.reset_index(drop=True, inplace=True)
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,bethel,41.3712,-73.4140,22.17,81,0,0.00,US,22-07-2023
32,32,ha'il,27.5219,41.6907,24.01,20,0,2.06,SA,22-07-2023
45,45,remire-montjoly,4.9167,-52.2667,24.02,100,0,0.51,GF,22-07-2023
46,46,aladag,37.5485,35.3960,24.86,39,0,2.46,TR,22-07-2023
99,99,rurrenabaque,-14.4413,-67.5278,21.07,78,0,0.90,BO,22-07-2023
126,126,dogubayazit,39.5469,44.0842,21.66,51,0,0.44,TR,22-07-2023
168,168,bainbridge,47.6262,-122.5212,24.75,71,0,2.57,US,22-07-2023
234,234,arish,31.1316,33.7984,26.13,62,0,1.66,EG,22-07-2023
279,279,pauini,-7.7136,-66.9764,22.87,60,0,0.34,BR,22-07-2023
301,301,asheboro,35.7079,-79.8136,24.20,77,0,0.00,US,22-07-2023


### Step 3: Create a new DataFrame called `hotel_df`.

In [100]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=pd.DataFrame()
hotel_df=city_data_df[["City", "Country", "Lng", "Lat", "Humidity"]].copy()
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()

,City,Country,Lng,Lat,Humidity,Hotel Name
6,bethel,US,-73.4140,41.3712,81,
32,ha'il,SA,41.6907,27.5219,20,
45,remire-montjoly,GF,-52.2667,4.9167,100,
46,aladag,TR,35.3960,37.5485,39,
99,rurrenabaque,BO,-67.5278,-14.4413,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [101]:
# Set parameters to search for a hotel
radius = 10000
limit=20
params = {
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    long = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    # your code here
    filters = f"circle:{long},{lat},{radius}"
    params["filter"] = filters
    bias = f"proximity:{long},{lat}"
    params["bias"] = bias
    
    # Adding hotel to params
    params["categories"]=f"accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    # YOUR CODE HERE
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    # YOUR CODE HERE
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
ha'il - nearest hotel: فندق كلاسيك
remire-montjoly - nearest hotel: Complexe Belova
aladag - nearest hotel: No hotel found
rurrenabaque - nearest hotel: Hostal Madidi
dogubayazit - nearest hotel: Yayla Palas
bainbridge - nearest hotel: Eagle Harbor Inn
arish - nearest hotel: سينا
pauini - nearest hotel: No hotel found
asheboro - nearest hotel: Fairfield Inn & Suites by Marriott Asheboro
hamilton - nearest hotel: North Vista Manor
tweed heads - nearest hotel: Mantra Twin Towers
gasa - nearest hotel: مسجد الفتح المبين
berdigestyakh - nearest hotel: No hotel found
greenville - nearest hotel: Quality Inn
zhezqazghan - nearest hotel: No hotel found
palmwoods - nearest hotel: Wildwood Sanctuary Bed & breakfast
kill devil hills - nearest hotel: Mariner Days Inn & Suites
riberalta - nearest hotel: Hotel Jomali
yevpatoriya - nearest hotel: Апогей
qurayyat - nearest hotel: دانة الشمال


,City,Country,Lng,Lat,Humidity,Hotel Name
6,bethel,US,-73.4140,41.3712,81,Hampton Inn Danbury
32,ha'il,SA,41.6907,27.5219,20,فندق كلاسيك
45,remire-montjoly,GF,-52.2667,4.9167,100,Complexe Belova
46,aladag,TR,35.3960,37.5485,39,No hotel found
99,rurrenabaque,BO,-67.5278,-14.4413,78,Hostal Madidi
126,dogubayazit,TR,44.0842,39.5469,51,Yayla Palas
168,bainbridge,US,-122.5212,47.6262,71,Eagle Harbor Inn
234,arish,EG,33.7984,31.1316,62,سينا
279,pauini,BR,-66.9764,-7.7136,60,No hotel found
301,asheboro,US,-79.8136,35.7079,77,Fairfield Inn & Suites by Marriott Asheboro


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [119]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
second_map =hotel_df.hvplot.points(
   "Lng",
   "Lat",
   hover_cols=["City", "Humidity", "Hotel Name", "Country"],
   geo=True,
   tiles="OSM",
   frame_width=800,
   frame_height=600,
   size="Humidity",
   color="City"
)
# Display the map
# YOUR CODE HERE
second_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)